# CAISO OASIS Data downloader

Data taken from [CAISO Oasis portal](http://oasis.caiso.com/mrioasis)

API documentation located [here](http://www.caiso.com/Documents/OASIS-InterfaceSpecification_v5_1_3Clean_Fall2017Release.pdf)

In [11]:
import requests
from selenium import webdriver
import pandas as pd
import zipfile as zp
import time
import os
import glob
import xml.etree.ElementTree as Xet

In [12]:
month_dict = {'Jan': ['0101', '0131'],
              'Feb': ['0201', '0228'],
              'Mar': ['0301', '0331'],
              'Apr': ['0401', '0430'],
              'May': ['0501', '0531'],
              'Jun': ['0601', '0630'],
              'Jul': ['0701', '0731'],
              'Aug': ['0801', '0831'],
              'Sep': ['0901', '0930'],
              'Oct': ['1001', '1031'],
              'Nov': ['1101', '1130'],
              'Dec': ['1201', '1231']
              }

month_dict['Dec'][0]

'1201'

In [13]:
def extract_monthly_data(year):
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': os.path.join(r"C:\Users\SESA639725\Documents\Git\CAISOprices\data", str(year))}
    chrome_options.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome(options=chrome_options)
    for month in month_dict.keys():
        api_call = "http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=" + \
            str(year) + month_dict[month][0] + "T07:00-0000&enddatetime=" + str(year) + month_dict[month][1] + \
                "T07:00-0000&version=1&resultformat=6&market_run_id=RTM&node=SLAP_SCEW-APND"
        print(api_call)
        driver.get(api_call)
        time.sleep(15)
        
    zip_files = glob.glob('../data/' + str(year) + '/*.zip')
    for zip_filename in zip_files:
        dir_name = os.path.splitext(zip_filename)[0]
        os.mkdir(dir_name)
        zip_handler = zp.ZipFile(zip_filename, "r")
        zip_handler.extractall(dir_name)

    # path = dir_name
    csv_files = glob.glob('../data/' + str(year) +'/*/*.csv')

    entries = []

    for csv in csv_files:
        entries.append(pd.read_csv(csv))

    combined_csvs = pd.concat(entries)
    combined_csvs.to_csv('../data/' + str(year) + '/' + str(year) + '_SCE_SLAPs.csv')

# extract_monthly_data(2023)

In [14]:
# Cleaning monthly data...
def clean_monthly_data(year):
    # Remove columns that are not needed...
    df = pd.read_csv('../data/' + str(year) + '/' + str(year) + '_SCE_SLAPs.csv')
    df = df.drop(df.columns[0], axis=1)
    df = df.drop(columns=['INTERVALENDTIME_GMT', 'OPR_DT', 'OPR_HR', 'NODE_ID_XML', 'NODE_ID', 'NODE', 'MARKET_RUN_ID', 'LMP_TYPE', 'PNODE_RESMRID', 'GRP_TYPE', 'POS', 'OPR_INTERVAL', 'GROUP'])

    # Rename columns...
    df = df.rename(columns={'MW': 'LMP'})

    # Convert 'LMP' to numeric...
    df['LMP'] = pd.to_numeric(df['LMP'])
    
    # Filter only for 'XML_DATA_ITEM' = 'LMP_ENE_PRC'...
    df = df[df['XML_DATA_ITEM'] == 'LMP_ENE_PRC']

    # Drop 'XML_DATA_ITEM' column...
    df = df.drop(columns=['XML_DATA_ITEM'])

    # Consolidate 5-minute data into 15-minute intervals...
    df['INTERVALSTARTTIME_GMT'] = pd.to_datetime(df['INTERVALSTARTTIME_GMT'])
    df = df.resample('15T', on='INTERVALSTARTTIME_GMT').mean()

    # Reset index...
    df = df.reset_index()

    # If the LMP is >= $200/MWh, make a new column 'High Price' and set to 1...
    df['High Price'] = 0
    df.loc[df['LMP'] >= 200, 'High Price'] = 1

    # Print total number of high price intervals...
    print('Total number of high price intervals in ' + str(year) + ': ' + str(df['High Price'].sum()))

    # Rename columns...
    df = df.rename(columns={'INTERVALSTARTTIME_GMT': 'datetime', 'LMP': 'price', 'High Price': 'merchant_schedule'})

    # Save cleaned data...
    df.to_csv('../data/' + str(year) + '/' + str(year) + '_SCE_SLAPs_cleaned.csv', index=False)

clean_monthly_data(2023)

Total number of high price intervals in 2023: 198


In [15]:
# def extract_monthly_5min_data(year, node):
#     chrome_options = webdriver.ChromeOptions()
#     prefs = {'download.default_directory' : os.path.join(r"C:\Users\SESA639725\Documents\Git\CAISOprices\data\5min", str(year))}
#     chrome_options.add_experimental_option('prefs', prefs)

#     driver = webdriver.Chrome(options=chrome_options)
#     for month in month_dict.keys():
#         api_call = "http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&resultformat=6&startdatetime=" + str(year)+month_dict[month][0] + "T07:00-0000&enddatetime=" +  str(year)+month_dict[month][1] + "T07:00-0000&version=1&market_run_id=HASP&node=" + node
#         print(api_call)
#         driver.get(api_call)
#         time.sleep(15)
        
#     zip_files = glob.glob('../data/5min/' +str(year)+'/*.zip')
#     for zip_filename in zip_files:
#         dir_name = os.path.splitext(zip_filename)[0]
#         os.mkdir(dir_name)
#         zip_handler = zp.ZipFile(zip_filename, "r")
#         zip_handler.extractall(dir_name)

#     # path = dir_name
#     csv_files = glob.glob('../data/5min/' + str(year) +'/*/*.csv')

#     entries =[]

#     for csv in csv_files:
#         entries.append(pd.read_csv(csv))

#     combined_csvs = pd.concat(entries)
#     combined_csvs.to_csv('../data/5min/' + str(year) +'/'+ str(year) + node +'.csv')

# extract_monthly_5min_data(2023, "CLEARKE_6_N012")

In [16]:
# def extract_monthly_15min_data(year, node):
#     chrome_options = webdriver.ChromeOptions()
#     prefs = {'download.default_directory' : '/Users/jaromeleslie/Documents/MDS/Personal_projects/CAISO_oasis_extractor/data/15min/'+str(year)}
#     chrome_options.add_experimental_option('prefs', prefs)

#     driver = webdriver.Chrome(chrome_options=chrome_options)
#     for month in month_dict.keys():
#         api_call = "http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&resultformat=6&startdatetime=" + str(year)+month_dict[month][0] + "T07:00-0000&enddatetime=" +  str(year)+month_dict[month][1] + "T07:00-0000&version=1&market_run_id=HASP&node=" + node
#         print(api_call)
#         driver.get(api_call)
#         time.sleep(15)
        
#     zip_files = glob.glob('../data/15min/' +str(year)+'/*.zip')
#     for zip_filename in zip_files:
#         dir_name = os.path.splitext(zip_filename)[0]
#         os.mkdir(dir_name)
#         zip_handler = zp.ZipFile(zip_filename, "r")
#         zip_handler.extractall(dir_name)

#     # path = dir_name
#     csv_files = glob.glob('../data/15min/' + str(year) +'/*/*.csv')

#     entries =[]

#     for csv in csv_files:
#         entries.append(pd.read_csv(csv))

#     combined_csvs = pd.concat(entries)
#     combined_csvs.to_csv('../data/15min/' + str(year) +'/'+ str(year) + node +'.csv')

# extract_monthly_15min_data(2019, "SLAP_SCEC-APND")

In [17]:
"http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_HASP_LMP&resultformat=6&startdatetime=20190101T07:00-0000&enddatetime=20190131T07:00-0000&version=1&market_run_id=HASP&node=SLAP_SCEC-APND"

'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_HASP_LMP&resultformat=6&startdatetime=20190101T07:00-0000&enddatetime=20190131T07:00-0000&version=1&market_run_id=HASP&node=SLAP_SCEC-APND'